In [2]:
import pandas as pd
import pickle
import os
from sqlalchemy import create_engine

In [4]:
engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

In [46]:
# Загрузка данных из таблицы user_data
user_data_query = "SELECT * FROM public.user_data"
user_data = pd.read_sql(user_data_query, engine)

In [47]:
user_data 

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [10]:
categorical_col = user_data.loc[:,user_data.dtypes==object].columns

In [12]:
numerical_col = user_data.loc[:,user_data.dtypes!=object].columns

In [13]:
user_data[categorical_col].nunique()

country      11
city       3915
os            2
source        2
dtype: int64

In [40]:
sng_countries = user_data['country'].value_counts().index[:4].to_list()

In [42]:
sng_countries

['Russia', 'Ukraine', 'Belarus', 'Kazakhstan']

In [48]:
user_data['sng_country'] = user_data['country'].apply(lambda x: 1 if x in sng_countries else 0)

In [50]:
top_50_cities = user_data['city'].value_counts().index[:50].to_list()

In [51]:
user_data['top_50_cities'] = user_data['city'].apply(lambda x: 1 if x in top_50_cities else 0)

In [52]:
user_data

,user_id,gender,age,country,city,exp_group,os,source,sng_country,top_50_cities
0,200,1,34,Russia,Degtyarsk,3,Android,ads,1,0
1,201,0,37,Russia,Abakan,0,Android,ads,1,0
2,202,1,17,Russia,Smolensk,4,Android,ads,1,0
3,203,0,18,Russia,Moscow,1,iOS,ads,1,1
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads,1,0
...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic,1,1
163201,168549,0,18,Russia,Tula,2,Android,organic,1,1
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic,1,1
163203,168551,0,38,Russia,Moscow,3,iOS,organic,1,1


In [53]:
for col in ['os', 'source']:
    one_hot = pd.get_dummies(user_data[col],prefix=col,drop_first=True)
    user_data = pd.concat((user_data.drop(col,axis=1),one_hot.astype(int)),axis=1)

In [56]:
user_data = user_data.drop(["city","country"], axis=1)

In [57]:
user_data

,user_id,gender,age,exp_group,sng_country,top_50_cities,os_iOS,source_organic
0,200,1,34,3,1,0,0,0
1,201,0,37,0,1,0,0,0
2,202,1,17,4,1,0,0,0
3,203,0,18,1,1,1,1,0
4,204,0,36,3,1,0,0,0
...,...,...,...,...,...,...,...,...
163200,168548,0,36,4,1,1,0,1
163201,168549,0,18,2,1,1,0,1
163202,168550,1,41,4,1,1,0,1
163203,168551,0,38,3,1,1,1,1


In [5]:
 # SQL queries to calculate features directly
active_query = """
    SELECT 
        user_id,
        SUM(CASE WHEN action = 'like' THEN 1 ELSE 0 END) AS number_of_likes
    FROM public.feed_data
    GROUP BY user_id
"""
active = pd.read_sql(active_query, engine)

In [17]:
import pandas as pd
from sqlalchemy import create_engine

def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def extract_features():
    # Создаем соединение с базой данных
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )

    # Загрузка данных из таблицы user_data
    user_data_query = "SELECT * FROM public.user_data"
    user_data = pd.read_sql(user_data_query, engine)

    # Преобразования
    sng_countries = user_data['country'].value_counts().index[:4].to_list()
    user_data['sng_country'] = user_data['country'].apply(lambda x: 1 if x in sng_countries else 0)
    user_data = user_data.drop('country', axis=1)
    
    top_50_cities = user_data['city'].value_counts().index[:50].to_list()
    user_data['top_50_cities'] = user_data['city'].apply(lambda x: 1 if x in top_50_cities else 0)
    user_data = user_data.drop('city', axis=1)
    
    for col in ['os', 'source']:
        one_hot = pd.get_dummies(user_data[col],prefix=col,drop_first=True)
        user_data = pd.concat((user_data.drop(col,axis=1),one_hot.astype(int)),axis=1)
    
    active_query = """
    SELECT 
        user_id,
        SUM(CASE WHEN action = 'like' THEN 1 ELSE 0 END) AS number_of_likes
    FROM public.feed_data
    GROUP BY user_id
    """
    active = pd.read_sql(active_query, engine)
    user_data = user_data.merge(active, on='user_id', how='left')
    
    # Create the active feature based on the number of likes
    user_data['active'] = user_data['number_of_likes'].apply(lambda x: 1 if x > 26 else 0)
    
    # Drop the like_actions column as it's no longer needed
    user_data = user_data.drop('number_of_likes', axis=1)

    # Запись объединенных данных в новую таблицу
    table_name = 'ivan_golov_features_lesson_22'
    user_data.to_sql(table_name, con=engine, if_exists='replace', index=False)

def load_features() -> pd.DataFrame:
    query = "SELECT * FROM ivan_golov_features_lesson_22"
    return batch_load_sql(query)

In [18]:
extract_features()
features = load_features()

In [20]:
features

,user_id,gender,age,exp_group,sng_country,top_50_cities,os_iOS,source_organic,active
0,200,1,34,3,1,0,0,0,1
1,201,0,37,0,1,0,0,0,1
2,202,1,17,4,1,0,0,0,1
3,203,0,18,1,1,1,1,0,1
4,204,0,36,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,4,1,1,0,1,0
163201,168549,0,18,2,1,1,0,1,0
163202,168550,1,41,4,1,1,0,1,1
163203,168551,0,38,3,1,1,1,1,1


In [21]:
# Загрузка данных из таблицы post_text_df
post_text_query = "SELECT * FROM public.post_text_df"
post_text_df = pd.read_sql(post_text_query, engine)

In [34]:
feed_data_query = "SELECT * FROM public.feed_data LIMIT 200000"
feed_data = pd.read_sql(feed_data_query, engine)

In [39]:
feed_data

,timestamp,user_id,post_id,action,target
0,2021-10-11 15:54:40,11613,2292,view,0
1,2021-10-11 15:56:23,11613,712,view,0
2,2021-10-11 15:56:36,11613,1222,view,0
3,2021-10-11 15:57:11,11613,3108,view,0
4,2021-10-11 15:59:46,11613,2123,view,0
...,...,...,...,...,...
199995,2021-11-03 09:27:24,19142,985,like,0
199996,2021-11-03 09:27:26,19142,6595,view,0
199997,2021-11-03 09:29:45,19142,1765,view,0
199998,2021-11-03 09:30:57,19142,6476,view,0


In [63]:
from datetime import datetime

def recommended_posts(user_id: int, timestamp: datetime, limit: int = 10) -> pd.DataFrame:
    # SQL query to get the recommended posts
    query = f"""
    SELECT fd.post_id, fd.user_id, fd.timestamp, fd.action, fd.target, pt.text, pt.topic
    FROM public.feed_data fd
    JOIN public.post_text_df pt ON fd.post_id = pt.post_id
    WHERE fd.user_id = {user_id} AND fd.timestamp >='{timestamp}'
    ORDER BY fd.timestamp DESC
    LIMIT {limit}
    """
    
    # Execute the query and load the data into a DataFrame
    posts_df = pd.read_sql(query, engine)
    
    return posts_df

In [64]:
# Example usage
user_id = 200
timestamp = datetime(2021, 10, 11, 15)
limit = 10

recommended_posts_df = recommended_posts(user_id, timestamp, limit)

In [65]:
recommended_posts_df = recommended_posts_df.merge(features, on='user_id')

In [66]:
recommended_posts_df 

,post_id,user_id,timestamp,action,target,text,topic,gender,age,exp_group,sng_country,top_50_cities,os_iOS,source_organic,active
0,1773,200,2021-12-29 15:24:59,view,0,Hearts 2-1 Livingston\n\nHearts wrapped up the...,sport,1,34,3,1,0,0,0,1
1,2213,200,2021-12-29 15:24:57,like,0,Warnings about junk mail deluge\n\nThe amount ...,tech,1,34,3,1,0,0,0,1
2,2213,200,2021-12-29 15:24:31,view,1,Warnings about junk mail deluge\n\nThe amount ...,tech,1,34,3,1,0,0,0,1
3,1122,200,2021-12-29 15:23:54,view,0,Errors doomed first Dome sale\n\nThe initial a...,politics,1,34,3,1,0,0,0,1
4,1362,200,2021-12-29 15:23:29,view,0,Bid to cut court witness stress\n\nNew targets...,politics,1,34,3,1,0,0,0,1
5,1541,200,2021-12-29 15:21:53,view,0,McIlroy aiming for Madrid title\n\nNorthern Ir...,sport,1,34,3,1,0,0,0,1
6,2160,200,2021-12-29 15:21:18,view,0,Reboot ordered for EU patent law\n\nA European...,tech,1,34,3,1,0,0,0,1
7,994,200,2021-12-29 15:21:16,like,0,Lib Dems highlight problem debt\n\nPeople vuln...,politics,1,34,3,1,0,0,0,1
8,994,200,2021-12-29 15:18:42,view,1,Lib Dems highlight problem debt\n\nPeople vuln...,politics,1,34,3,1,0,0,0,1
9,1670,200,2021-12-29 15:16:48,view,0,Federer claims Dubai crown\n\nWorld number one...,sport,1,34,3,1,0,0,0,1
